# 1) 라이브러리 임포트 / Import Libraries
**[KOR]** 이 셀에서는 전체 파이프라인에 필요한 라이브러리를 임포트합니다.
**[ENG]** This cell imports all libraries required for the entire pipeline.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

# Hugging Face 관련 라이브러리 / Hugging Face libraries
from transformers import (ViTImageProcessor, ViTForImageClassification,
                          TFViTModel, create_optimizer, TrainingArguments, Trainer,
                          DefaultDataCollator)

# W&B / Weights & Biases
import wandb
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

# 기타 / Others
import numpy as np
import os
from datasets import Dataset

# 2) W&B 프로젝트 초기화 / Initialize W&B
**[KOR]** 여기서는 W&B 프로젝트를 초기화합니다. 원하는 프로젝트 이름과 실험명을 지정할 수 있습니다.
**[ENG]** Here, we initialize a W&B project. Customize the project name and run name as desired.

In [2]:
wandb.init(
    project="ViT_vs_CNN_jellyfish", 
    name="Initial_Run", 
    reinit=True
)

wandb: Currently logged in as: realx1212 (realx1212-aijosh). Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


# 3) 데이터셋 로드 및 전처리 / Load and Preprocess the Dataset
**[KOR]** 이 부분만 기존 tf_flowers 대신, Google Drive 경로에 있는 local dataset(jellyfish 등)을 로드하는 코드로 변경합니다. 
**[ENG]** This section is modified to load a local dataset (e.g., jellyfish) from a Google Drive path instead of tf_flowers.

In [3]:
import os
import tensorflow as tf

data_path = r"C:\Users\User\Documents\GitHub\National_OceanoGraphic\dataset\3_classweight80"
batch_size = 32
img_size = (224, 224)

# train/ val/ test/ 디렉토리가 이미 존재한다고 가정
train_dir = os.path.join(data_path, 'train')
val_dir = os.path.join(data_path, 'val')
test_dir = os.path.join(data_path, 'test')

def preprocess_image_ds(ds):
    # 데이터 증강, 정규화를 진행하지만, 여기서는 배치 X
    ds = ds.map(lambda x, y: (tf.image.resize(x, img_size), y),
                num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.map(lambda x, y: (x / 255.0, y),
                num_parallel_calls=tf.data.AUTOTUNE)
    # 배치와 prefetch는 image_dataset_from_directory에서 이미 배치되었으므로 제거
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds

# 이미 batch_size=32로 dataset을 생성
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    label_mode='int',
    image_size=img_size,
    batch_size=batch_size
)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    val_dir,
    label_mode='int',
    image_size=img_size,
    batch_size=batch_size
)
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    label_mode='int',
    image_size=img_size,
    batch_size=batch_size
)

# 이제 preprocess_image_ds에서는 배치를 다시 하지 않음
train_batches = preprocess_image_ds(train_ds)
val_batches   = preprocess_image_ds(val_ds)
test_batches  = preprocess_image_ds(test_ds)

# 클래스 개수 파악
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)
print("num_classes:", num_classes)

# shape 확인 (예시)
for images, labels in train_batches.take(1):
    print("train batch shape:", images.shape)  # (32, 224, 224, 3) 형태가 되어야 함

for i, (images, labels) in enumerate(train_batches):
    print(f"Batch {i} shape:", images.shape)
    # 필요하다면 라벨 shape도 확인
    print(f"Labels {i} shape:", labels.shape)
    
    # 10개 배치만 확인 (예시)
    if i >= 9:
        break

for i, (images, labels) in enumerate(val_batches):
    print(f"Val Batch {i} shape:", images.shape)
    if i >= 2:
        break

for i, (images, labels) in enumerate(test_batches):
    print(f"Test Batch {i} shape:", images.shape)
    if i >= 2:
        break

Found 427 files belonging to 6 classes.
Found 51 files belonging to 6 classes.
Found 56 files belonging to 6 classes.
Classes: ['barrel_jellyfish', 'blue_jellyfish', 'compass_jellyfish', 'lions_mane_jellyfish', 'mauve_stinger_jellyfish', 'moon_jellyfish']
num_classes: 6
train batch shape: (32, 224, 224, 3)
Batch 0 shape: (32, 224, 224, 3)
Labels 0 shape: (32,)
Batch 1 shape: (32, 224, 224, 3)
Labels 1 shape: (32,)
Batch 2 shape: (32, 224, 224, 3)
Labels 2 shape: (32,)
Batch 3 shape: (32, 224, 224, 3)
Labels 3 shape: (32,)
Batch 4 shape: (32, 224, 224, 3)
Labels 4 shape: (32,)
Batch 5 shape: (32, 224, 224, 3)
Labels 5 shape: (32,)
Batch 6 shape: (32, 224, 224, 3)
Labels 6 shape: (32,)
Batch 7 shape: (32, 224, 224, 3)
Labels 7 shape: (32,)
Batch 8 shape: (32, 224, 224, 3)
Labels 8 shape: (32,)
Batch 9 shape: (32, 224, 224, 3)
Labels 9 shape: (32,)
Val Batch 0 shape: (32, 224, 224, 3)
Val Batch 1 shape: (19, 224, 224, 3)
Test Batch 0 shape: (32, 224, 224, 3)
Test Batch 1 shape: (24, 224, 

# 4) 간단 CNN 모델 정의 / Define a Simple CNN Model
**[KOR]** 전통적인 CNN 모델과 비교하기 위해, 간단한 Conv-BN-MaxPool 구조의 CNN 모델을 정의합니다.
**[ENG]** Define a simple CNN model (Conv-BN-MaxPool architecture) for comparison with the ViT model.

In [4]:
def create_simple_cnn(num_classes):
    inputs = tf.keras.Input(shape=(224, 224, 3), name="input_images")
    x = tf.keras.layers.Conv2D(32, 3, activation='relu')(inputs)
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
    x = tf.keras.layers.MaxPool2D()(x)
    x = tf.keras.layers.Conv2D(128, 3, activation='relu')(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)
    return model

# 5) CNN 모델 훈련 / Train the CNN Model
**[KOR]** 간단 CNN 모델을 학습시키고 W&B에 로깅합니다. CNN 성능을 파악한 뒤, 이후 ViT와 비교할 수 있습니다.
**[ENG]** Train the simple CNN model and log metrics to W&B. We can then compare its performance to the ViT model.

In [14]:
# CNN model
cnn_model = create_simple_cnn(num_classes)

wandb.init(
    project="ViT_vs_CNN_jellyfish", 
    name="CNN_flowers_run", 
    reinit=True
)

cnn_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

cnn_model.fit(
    train_batches,
    validation_data=val_batches,
    epochs=10,  # 예시로 3 epoch
     callbacks=[WandbMetricsLogger(log_freq="epoch"),WandbModelCheckpoint("model_checkpoint.keras")] 
)

# Evaluate on test set
cnn_eval = cnn_model.evaluate(test_batches)
print("CNN Test Evaluation:", cnn_eval)
wandb.finish()

Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.2500 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 4s 412ms/step - accuracy: 0.2578 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 4s 416ms/step - accuracy: 0.2413 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 4s 418ms/step - accuracy: 0.2279 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 3s 435ms/step - accuracy: 0.2223 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 3s 435ms/step - accuracy: 0.2174 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 3s 436ms/step - accuracy: 0.2150 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 2s 433ms/step - accuracy: 0.2121 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 2s 432ms/step - accuracy: 0.2086 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 1s 433ms/step - accuracy: 0.2061 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 1s 436ms/step - accuracy: 0.2050 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 0s 444ms/step - accuracy: 0.2050 - loss: 1.78 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step - accuracy: 0.2042 - loss: 1.77 ━━━━━━━━━━━━━━━━━━━━ 0s 418ms/step - accuracy: 0.2034 - loss: 1.77 ━━━━━━━━━━━━━━━━━━━━ 8s 467ms/step - accuracy: 

epoch/accuracy,▁▂▃▅▅▄▆▆▆█
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▅▄▄▃▃▂▁
epoch/val_accuracy,▁▃▆▆▅▆▆██▇
epoch/val_loss,█▇▅▄▅▃▁▂▁▁
epoch/accuracy,0.46838
epoch/epoch,9
epoch/learning_rate,0.001
epoch/loss,1.40162
epoch/val_accuracy,0.45098


In [25]:
import tensorflow as tf
from transformers import TFViTModel

class TFPixelValuesWrapper(tf.keras.layers.Layer):
    def __init__(
        self,
        model_name="google/vit-base-patch16-224-in21k",
        base_trainable=False,
        **kwargs
    ):
        super().__init__(**kwargs)
        # 사전 학습된 TFViTModel 로드
        self.base_vit = TFViTModel.from_pretrained(model_name)
        self.base_vit.trainable = base_trainable
        
        self.hidden_size = self.base_vit.config.hidden_size
        self.patch_size  = self.base_vit.config.patch_size
        self.image_size  = self.base_vit.config.image_size

    def call(self, inputs, training=False):
        """
        inputs: (batch, 224, 224, 3)
        TFViTModel expects pixel_values in (batch, channels, height, width) or just a normal tf.Tensor.
        """
        # KerasTensor -> 실제 텐서 변환 + transpose (channels-last → channels-first)
        pixel_values = tf.transpose(inputs, perm=[0, 3, 1, 2])  # (B, 3, 224, 224)

        outputs = self.base_vit(pixel_values=pixel_values, training=training)
        # 예: outputs.last_hidden_state (B, seq_len, hidden_dim)
        return outputs.last_hidden_state

    def compute_output_shape(self, input_shape):
        # (batch, 224, 224, 3)
        # seq_len = (224/patch_size)*(224/patch_size)
        batch_size = input_shape[0]
        seq_len = (self.image_size // self.patch_size) * (self.image_size // self.patch_size)
        return (batch_size, seq_len, self.hidden_size)


# 6) Functional API 기반 ViT 모델 빌드 함수 / Build ViT Model with Functional API
**[KOR]** 사전 학습된 ViT를 불러와, Dense와 Dropout, BatchNorm 등을 파라미터로 동적으로 추가/조절하는 함수를 만듭니다.
**[ENG]** Load a pretrained ViT, then add Dense, Dropout, and BatchNorm layers dynamically based on hyperparameters in a build function.

In [11]:
def build_vit_model(
    num_classes,
    num_dense_layers=1,
    dense_units=128,
    dropout_rate=0.3,
    use_batchnorm=True,
    base_trainable=False
):
    # (224, 224, 3) 입력
    inputs = tf.keras.Input(shape=(224, 224, 3), name="input_images")

    # 래퍼 레이어 인스턴스
    vit_layer = TFPixelValuesWrapper(
        model_name="google/vit-base-patch16-224-in21k",
        base_trainable=base_trainable
    )

    # ViT 출력: (batch, seq_len, hidden_dim)
    vit_outputs = vit_layer(inputs)

    # seq_len 차원 풀링
    x = tf.keras.layers.GlobalAveragePooling1D()(vit_outputs)

    # 추가 Dense 레이어 쌓기
    for _ in range(num_dense_layers):
        x = tf.keras.layers.Dense(dense_units, activation='relu')(x)
        if use_batchnorm:
            x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Dropout(dropout_rate)(x)

    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    return tf.keras.Model(inputs=inputs, outputs=outputs)


# 7) ViT 모델 학습 (단일 실험 예시) / Train the ViT Model (Single Experiment Example)
**[KOR]** 위에서 정의한 `build_vit_model`을 사용해 ViT 모델을 하나 구성해 보고, W&B에 로깅합니다.
**[ENG]** Use `build_vit_model` to create a ViT model, then train and log metrics to W&B as a single-run example.

In [15]:
wandb.init(
    project="ViT_vs_CNN_jellyfish",
    name="ViT_single_run",
    reinit=True
)

vit_model = build_vit_model(
    num_classes=num_classes,
    num_dense_layers=2,
    dense_units=128,
    dropout_rate=0.3,
    use_batchnorm=True,
    base_trainable=False
)

vit_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

vit_model.fit(
    train_batches,
    validation_data=val_batches,
    epochs=10,
     callbacks=[WandbMetricsLogger(log_freq="epoch"),WandbModelCheckpoint("model_checkpoint.keras")] 
)

test_loss, test_acc = vit_model.evaluate(test_batches)
wandb.log({"test_loss": test_loss, "test_accuracy": test_acc})
print("ViT Test Accuracy:", test_acc)
wandb.finish()

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


Epoch 1/10
14/14 ━━━━━━━━━━━━━━━━━━━━ 2:40 12s/step - accuracy: 0.0625 - loss: 3.10 ━━━━━━━━━━━━━━━━━━━━ 46s 4s/step - accuracy: 0.0859 - loss: 3.0128 ━━━━━━━━━━━━━━━━━━━━ 42s 4s/step - accuracy: 0.1024 - loss: 2.94 ━━━━━━━━━━━━━━━━━━━━ 38s 4s/step - accuracy: 0.1217 - loss: 2.87 ━━━━━━━━━━━━━━━━━━━━ 35s 4s/step - accuracy: 0.1361 - loss: 2.81 ━━━━━━━━━━━━━━━━━━━━ 31s 4s/step - accuracy: 0.1534 - loss: 2.74 ━━━━━━━━━━━━━━━━━━━━ 27s 4s/step - accuracy: 0.1691 - loss: 2.68 ━━━━━━━━━━━━━━━━━━━━ 23s 4s/step - accuracy: 0.1831 - loss: 2.62 ━━━━━━━━━━━━━━━━━━━━ 19s 4s/step - accuracy: 0.1944 - loss: 2.57 ━━━━━━━━━━━━━━━━━━━━ 15s 4s/step - accuracy: 0.2043 - loss: 2.53 ━━━━━━━━━━━━━━━━━━━━ 11s 4s/step - accuracy: 0.2139 - loss: 2.49 ━━━━━━━━━━━━━━━━━━━━ 7s 4s/step - accuracy: 0.2230 - loss: 2.4547 ━━━━━━━━━━━━━━━━━━━━ 3s 4s/step - accuracy: 0.2315 - loss: 2.419 ━━━━━━━━━━━━━━━━━━━━ 0s 4s/step - accuracy: 0.2393 - loss: 2.388 ━━━━━━━━━━━━━━━━━━━━ 69s 4s/step - accuracy: 0.2460 - loss: 2.3615 -

epoch/accuracy,▁▅▆▇▇█████
epoch/epoch,▁▂▃▃▄▅▆▆▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▄▃▂▂▁▁▁▁▁
epoch/val_accuracy,▁▄▅▆██▇█▇█
epoch/val_loss,█▇▆▅▄▄▃▂▁▁
test_accuracy,▁
test_loss,▁
epoch/accuracy,0.93677
epoch/epoch,9
epoch/learning_rate,0.001


# 8) W&B Sweep 설정 예시 (sweep.yaml) / W&B Sweep Configuration Example
**[KOR]** 다양한 하이퍼파라미터(레이어 수, Dropout 비율, BatchNorm 여부 등)를 탐색하기 위한 예시 스윕 설정입니다.
**[ENG]** An example sweep configuration for exploring various hyperparameters (number of Dense layers, dropout rate, batch normalization, etc.).

In [20]:
sweep_config = '''
method: bayes
metric:
  name: val_accuracy
  goal: maximize
parameters:
  num_dense_layers:
    values: [2, 3]
  dense_units:
    values: [64, 128, 256]
  dropout_rate:
    values: [0.2, 0.3, 0.5]
  use_batchnorm:
    values: [true, false]
  base_trainable:
    values: [false, true]
  learning_rate:
    values: [1e-2, 1e-3, 1e-4]
  batch_size:
    values: [16, 32]
  epochs:
    values: [5, 10]
'''

print(sweep_config)


method: bayes
metric:
  name: val_accuracy
  goal: maximize
parameters:
  num_dense_layers:
    values: [2, 3]
  dense_units:
    values: [64, 128, 256]
  dropout_rate:
    values: [0.2, 0.3, 0.5]
  use_batchnorm:
    values: [true, false]
  base_trainable:
    values: [false, true]
  learning_rate:
    values: [1e-2, 1e-3, 1e-4]
  batch_size:
    values: [16, 32]
  epochs:
    values: [5, 10]



# 9) Sweep 트레이닝 함수 / Sweep Training Function
**[KOR]** Sweep 에이전트가 각 실험마다 호출할 함수입니다. 여러 조합을 자동 시도하며 결과를 W&B에 기록합니다.
**[ENG]** This function is called by the sweep agent for each experiment, automatically trying different combinations and logging results to W&B.

In [26]:
def train_sweep(config=None):
    with wandb.init(config=config):
        config = wandb.config

        # Ensure batch_size is set correctly
        current_train_batches = train_ds.batch(config.batch_size).prefetch(tf.data.AUTOTUNE)
        current_val_batches = val_ds.batch(config.batch_size).prefetch(tf.data.AUTOTUNE)
        current_test_batches = test_ds.batch(config.batch_size).prefetch(tf.data.AUTOTUNE)

        model = build_vit_model(
            num_classes=num_classes,
            num_dense_layers=config.num_dense_layers,
            dense_units=config.dense_units,
            dropout_rate=config.dropout_rate,
            use_batchnorm=config.use_batchnorm,
            base_trainable=config.base_trainable
        )

        optimizer = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
        model.compile(
            optimizer=optimizer,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        model.fit(
            current_train_batches,
            validation_data=current_val_batches,
            epochs=config.epochs,
            callbacks=[WandbMetricsLogger(log_freq="epoch"), WandbModelCheckpoint("model_checkpoint.keras")]
        )

        test_loss, test_acc = model.evaluate(current_test_batches)
        wandb.log({"test_loss": test_loss, "test_accuracy": test_acc})
        print("Test Accuracy:", test_acc)

# 10) Sweep 실행 / Launch the Sweep
**[KOR]** 아래 코드(또는 CLI 명령어)를 통해 sweep을 생성하고, 에이전트를 실행하면 다양한 아키텍처 및 HP 조합이 자동으로 탐색됩니다.
**[ENG]** Use the code (or CLI commands) below to create a sweep and run an agent, automatically exploring multiple architecture and hyperparameter combinations.

In [27]:
# (주의) 실제로 실행할 때 주석 해제
import yaml
sweep_config_dict = yaml.safe_load(sweep_config)
sweep_id = wandb.sweep(sweep_config_dict, project="ViT_jellyfish_HPtuing")
wandb.agent(sweep_id, function=train_sweep)

Create sweep with ID: 7jsde31a
Sweep URL: https://wandb.ai/realx1212-aijosh/ViT_jellyfish_HPtuing/sweeps/7jsde31a


wandb: Agent Starting Run: 6d46smmx with config:
wandb: 	base_trainable: True
wandb: 	batch_size: 16
wandb: 	dense_units: 128
wandb: 	dropout_rate: 0.5
wandb: 	epochs: 5
wandb: 	learning_rate: 0.001
wandb: 	num_dense_layers: 3
wandb: 	use_batchnorm: True


All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


Epoch 1/5


Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_27064\1046017399.py", line 26, in train_sweep
    model.fit(
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\input_spec.py", line 245, in assert_input_compatibility
    raise ValueError(
ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, None, 224, 224, 3)


...

Run 9dkkkbz1 errored:
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\User\AppData\Local\Temp\ipykernel_27064\1046017399.py", line 26, in train_sweep
    model.fit(
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\input_spec.py", line 245, in assert_input_compatibility
    raise ValueError(
ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, None, 224, 224, 3)

wandb: ERROR Run 9dkkkbz1 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\si

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


Epoch 1/5


Traceback (most recent call last):
  File "C:\Users\User\AppData\Local\Temp\ipykernel_27064\1046017399.py", line 26, in train_sweep
    model.fit(
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\input_spec.py", line 245, in assert_input_compatibility
    raise ValueError(
ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, None, 224, 224, 3)


Run pd1ttb82 errored:
Traceback (most recent call last):
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\wandb\agents\pyagent.py", line 306, in _run_job
    self._function()
  File "C:\Users\User\AppData\Local\Temp\ipykernel_27064\1046017399.py", line 26, in train_sweep
    model.fit(
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\input_spec.py", line 245, in assert_input_compatibility
    raise ValueError(
ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 224, 224, 3), found shape=(None, None, 224, 224, 3)

wandb: ERROR Run pd1ttb82 errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "c:\Users\User\AppData\Local\Programs\Python\Python310\lib\si

# 요약 / Summary
**[KOR]** 위 노트북을 통해 전통적인 CNN과 사전 학습된 ViT 기반 모델을 모두 학습해보고, W&B의 Sweep 기능으로 아키텍처 및 하이퍼파라미터를 자동 탐색할 수 있습니다.
**[ENG]** This notebook allows you to train both a traditional CNN and a pretrained ViT model, and use W&B Sweeps to automatically search for optimal architectures and hyperparameters.